In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.6 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
     |████████████████████████████████| 120 kB 72.7 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_name = '/content/drive/My Drive/quote_recommendation'

In [5]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [6]:
# Import generic wrappers
from transformers import AutoTokenizer 
# Define the model repo
model_name = "SpanBERT/spanbert-base-cased" 
tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)

In [7]:
import json
with open("/content/drive/My Drive/quote_recommendation/gandhi.json", "r") as read_it:
     data = json.load(read_it)

list_mp_val = []
for key,val in data.items():
    temp = {
    "answers":
        {
            "text":"",
            "answer_start":0,
            "answer_end":0
        },
    "context":"",
    "questions":""
    }
    temp["answers"]["text"] = val["quote"]
    temp["answers"]["answer_start"] = val["start_index"]
    temp["answers"]["answer_end"] = val["end_index"]
    temp["questions"] = key
    temp["context"] = val["paragraph"]
    list_mp_val.append(temp)

  
def read_quotus(list_mp):
    contexts = []
    questions = []
    answers = []
    for val in list_mp:
        contexts.append(val["context"])
        questions.append(val["questions"])
        answers.append(val["answers"])
    return contexts, questions, answers


val_contexts, val_questions, val_answers = read_quotus(list_mp_val)

In [8]:
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [9]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
answers_starts = []
answers_ends = []
actual_starts , actual_ends = [],[]
#n = len(val_questions)
for id in range(50):
    question = val_questions[id]
    context = val_contexts[id]

    actual_starts.append([val_answers[id]["answer_start"]])
    actual_ends.append([val_answers[id]["answer_end"]])

    inputs = tokenizer.encode_plus(question, context, return_tensors='pt',max_length=512)
    #inputs.to(device)
    outputs = model(**inputs)

    # print(outputs.__dict__)
    print(f"================================================================================================================")
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    #print(f"answer start {answer_start}")
    answers_starts.append(torch.topk(outputs[0], 5)[1][0])
    #print(f"predicted answer starts {answers_starts}")
    #print(f"actual answer starts {actual_starts}")
    answer_end = torch.argmax(outputs[1])
    #print(f"answer end {answer_end}")
    answers_ends.append(torch.topk(outputs[1], 5)[1][0])
    #print(f"answer ends {answers_ends}")
    #print(f"actual answer ends {actual_ends}")
    print("Actual result ------------------------------")
    actual_answer = val_answers[id]["text"]
    print(actual_answer)
    #print(context[answer_start:answer_end+1])
    print(f"============================================================")
    print("Top 5 predicted results")
    for i in range(5):
      print(f"pridicted answer {i+1} -> {tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answers_starts[id][i]:answers_ends[id][i]+1]))}")
      #print(f"pridicted answer {i+1} -> {context[answers_starts[id][i]:answers_ends[id][i]+1]}")


Actual result ------------------------------
In this world there are only two tragedies. One is not getting what one wants, and the other is getting it.
Top 5 predicted results
pridicted answer 1 -> i wish you good luck,
pridicted answer 2 -> 
pridicted answer 3 -> 
pridicted answer 4 -> 
pridicted answer 5 -> 
Actual result ------------------------------
To live is the rarest thing in the world. Most people exist, that is all.
Top 5 predicted results
pridicted answer 1 -> most people exist,
pridicted answer 2 -> nobody will waste their life accumulating and symbols for things. [SEP] with the abolition of private property, then, we shall have true, beautiful, healthy individualism. nobody will waste his life in accumulating things, and the symbols for things. one will live. to live is the rarest thing in the world.
pridicted answer 3 -> accumulating and symbols for things.
pridicted answer 4 -> with the abolition of private property, then, we shall have true, beautiful, healthy individ